## 1 - Importar Bibliotecas e estabelecer o diretório atual

In [ ]:
import pandas as pd 
import requests
import os
import zipfile
import numpy as np

pd.set_option('display.max_columns', None)

diretorio_atual = os.getcwd()

os.chdir(f"{diretorio_atual}\dados_cvm")

# 2 - Download dos Arquivos da CVM

In [ ]:
# Download dos arquivos dentro do range

anos = range(2017, 2023)

url_base = "https://dados.cvm.gov.br/dados/CIA_ABERTA/DOC/DFP/DADOS/"

for ano in anos:
    
    download = requests.get(url_base + f"dfp_cia_aberta_{ano}.zip")
    open(f"dfp_cia_aberta_{ano}.zip", "wb").write(download.content)

# 3 - Criação da lista de todas as base de dados

In [ ]:
# Criação da lista de todas as  base de dados


lista_demonstracoes_2010_2021 = []

diretorio_atual = os.getcwd()

for arquivo in os.listdir(diretorio_atual):
    
    
    if arquivo == ".ipynb_checkpoints":
        pass
    else:
        arquivo_zip = zipfile.ZipFile(arquivo)
            
        for planilha in arquivo_zip.namelist(): 
        
                demonstracao = pd.read_csv(arquivo_zip.open(planilha), sep = ";", encoding = 'ISO-8859-1', 
                                       dtype = {"ORDEM_EXERC": "category"})

                lista_demonstracoes_2010_2021.append(demonstracao)

# 4 - Criar um Dataframe com os dados

In [ ]:
# concatenar as listas em um único dataframe
base_dados = pd.concat(lista_demonstracoes_2010_2021)

#df.loc[(df['coluna1'] == 'valor1') & (df['coluna2'] == 'valor2'), 'coluna3']

# arrumar o dataframe

base_dados[['con_ind', 'tipo_dem']] = base_dados['GRUPO_DFP'].str.split("-", expand = True)
base_dados['con_ind'] = base_dados['con_ind'].str.strip()
base_dados['tipo_dem'] = base_dados['tipo_dem'].str.strip()
base_dados['CD_CONTA'] = base_dados['CD_CONTA'].str.strip()

base_dados = base_dados[base_dados['ORDEM_EXERC'] != "PENÚLTIMO"]


#Certificar-se de que não existem linhas repetidas:

base_dados = base_dados.drop_duplicates()

# 5 - Criar um dataframe com apenas as empresas Cíclicas

In [ ]:
#Identificar as empresas Ciclicas através de uma planilha de classificação. A planilha
#"Classificador" possui a classificação do segmento ou setor das empresas

classificador = pd.read_excel("/Users/gerso/OneDrive/diversos antigos/Documentos/usp/00 - PROJETO TCC/ProgramaJupyter/Classificacao.xlsx")


# Fazer um procv para as categorias serem incluidas à base principal.
# Mantido as mudanças na mesma base de dados
base_dados = pd.merge(base_dados, classificador, how='left', left_on='DENOM_CIA', right_on='NOME_CVM')

# Apagar todas as linhas que não possuem a descrição da empresa b3 (todos que não são ciclicos na categoria)
base_dados = base_dados.dropna(subset=["EMPRESA B3"])
base_dados = base_dados.dropna(subset=["CD_CONTA"])

### Agregar a base de dados por valor da conta sendo o maior absoluto 
colunas_escolhidas = ['CODIGOB3','DENOM_CIA','CNPJ_CIA','SUBSETOR','SEGMENTO', 'DT_FIM_EXERC','DS_CONTA','CD_CONTA', 'VL_CONTA']

colunas_escolhidas2 = ['DENOM_CIA','DT_FIM_EXERC','CD_CONTA']

base = base_dados.copy()

base = base.loc[:, colunas_escolhidas]

base['VL_CONTA2'] = base.groupby(colunas_escolhidas2)['VL_CONTA'].transform(lambda x: max(x, key=abs))


# Igualar as duas contas para depois excluir a duplicada

base['VL_CONTA'] = base['VL_CONTA2']


# excluir uma das contas e renomear novamente a outra pelo padrão

base = base.drop('VL_CONTA', axis=1)
base = base.rename(columns={'VL_CONTA2': 'VL_CONTA'})

#Retirar os dados duplicados

base = base.drop_duplicates()

#Zerar os índices

base = base.reset_index(drop=True)

base

# 6 - FUNÇÃO  Valor_conta para RETORNAR O VALOR DA CONTA

In [ ]:
#Função de Filtro para retornar o valor da conta usado no calculo dos indicadores seguintes


def Valor_conta(df, data_ref, Denom_Cia, cd_conta):
    filtro = (df['DENOM_CIA'] == Denom_Cia) & (df['CD_CONTA'] == cd_conta) & (df['DT_FIM_EXERC'] == data_ref)
    df_filtrado = df.loc[filtro]
    
    # Retornar o valor único da coluna especificada
    if len(df_filtrado) > 0:
        x = df_filtrado['VL_CONTA'].unique()[0]
    else:
        x = None
    return x

# 9 - Indicadores de Liquidez - Funções

In [ ]:
#Função do Indicador de Liquidez Corrente:

def Liquidez_corrente(df, data_ref, Denom_Cia):
    ativo_circulante = Valor_conta(df, data_ref, Denom_Cia, '1.01')
    passivo_circulante = Valor_conta(df, data_ref, Denom_Cia, '2.01')
    
    if ativo_circulante is None or passivo_circulante is None or passivo_circulante == 0:
        return None
    return ativo_circulante/passivo_circulante


#Função de Liquidez Seca (LS)

def Liquidez_seca(df, data_ref, Denom_Cia):
    Ativo_Circulante = Valor_conta(df, data_ref, Denom_Cia, '1.01')
    Estoque = Valor_conta(df, data_ref, Denom_Cia, '1.01.04')
    
    
    #Despesas antecipadas diversas:
    Despa1 = Valor_conta(df, data_ref, Denom_Cia, '1.01.07')
    Despa2 = Valor_conta(df, data_ref, Denom_Cia, '1.01.07.01') 
    Despa3 = Valor_conta(df, data_ref, Denom_Cia, '1.01.07.02')
    if Despa1 is not None:
        Despesas_antecipadas = Despa1
    elif Despa2 is not None and Despa3 is None:
        Despesas_antecipadas = Despa2
    elif Despa3 is not None and Despa2 is None:
        Despesas_antecipadas = Despa3
    elif Despa3 is None and Despa2 is None:
        Despesas_antecipadas = 0
    else: Despesas_antecipadas = Despa2+Despa3

    
    Passivo_Circulante = Valor_conta(df, data_ref, Denom_Cia, '2.01')

    if Ativo_Circulante is None or Estoque is None or Despesas_antecipadas is None or Passivo_Circulante is None or Passivo_Circulante == 0:
        return None
    return (Ativo_Circulante-Estoque-Despesas_antecipadas)/Passivo_Circulante



#Função de Liquidez Geral (LG)

def Liquidez_geral(df, data_ref, Denom_Cia):
    Ativo_Circulante = Valor_conta(df, data_ref, Denom_Cia, '1.01')
    Passivo_Circulante = Valor_conta(df, data_ref, Denom_Cia, '2.01')
    Real_Long_Praz = Valor_conta(df, data_ref, Denom_Cia, '1.02.01')
    Ex_Long_Praz = Valor_conta(df, data_ref, Denom_Cia, '2.02')


    if (Ativo_Circulante is None or Passivo_Circulante is None or Real_Long_Praz is None or Ex_Long_Praz is None) or ((Passivo_Circulante + Ex_Long_Praz) == 0):
        return None
    return ((Ativo_Circulante+Real_Long_Praz)/(Passivo_Circulante+Ex_Long_Praz))



# Função de Liquidez Imediata (LI)

def liquidez_Imediata(df, data_ref, Denom_Cia):

 # Filtrar por Passivo_Circulante
    Passivo_Circulante= Valor_conta(df, data_ref, Denom_Cia, '2.01')
    Disponivel = Valor_conta(df, data_ref, Denom_Cia, '1.01.01')
   
    if Disponivel is None or Passivo_Circulante is None or Passivo_Circulante == 0:
        return None
    return Disponivel/Passivo_Circulante

# Indicadores de Endividamento - Funções 

In [ ]:
#*****************Relação de Capital de terceiro por Capital Próprio***********************************************

def CapTer_CapPro(df, data_ref, Denom_Cia): # Obs.  Quanto maior, provavelmente pior (verificar de ajustar esse)
    Passivo_Circulante = Valor_conta(df, data_ref, Denom_Cia, '2.01')
    Ex_Long_Praz = Valor_conta(df, data_ref, Denom_Cia, '2.02')
    Patr_Liq = Valor_conta(df, data_ref, Denom_Cia, '2.03')
    
    if Passivo_Circulante is None or Ex_Long_Praz is None or Patr_Liq is None or Patr_Liq == 0:
        return None
    return (Passivo_Circulante + Ex_Long_Praz)/Patr_Liq


#*****************Relação de Capital de terceiro sobre Passivo Total ***********************************************

def CapTer_PassTot(df, data_ref, Denom_Cia):
    Passivo_Circulante = Valor_conta(df, data_ref, Denom_Cia, '2.01')
    Ex_Long_Praz = Valor_conta(df, data_ref, Denom_Cia, '2.02')
    Patr_Liq = Valor_conta(df, data_ref, Denom_Cia, '2.03')
 
    if Passivo_Circulante is None or Ex_Long_Praz is None or Patr_Liq is None or (Patr_Liq+Passivo_Circulante+Ex_Long_Praz) == 0:
        return None
    return (Passivo_Circulante + Ex_Long_Praz)/(Patr_Liq+Passivo_Circulante+Ex_Long_Praz)


    
#*****************Imobilização dos Recursos Permanentes***********************************************


def Imob_RPer(df, data_ref, Denom_Cia):
    Ex_Long_Praz = Valor_conta(df, data_ref, Denom_Cia, '2.02')
    At_Perm = Valor_conta(df, data_ref, Denom_Cia, '1.02')
    Patr_Liq = Valor_conta(df, data_ref, Denom_Cia, '2.03')
 
    if Ex_Long_Praz is None or At_Perm is None or Patr_Liq is None or (Patr_Liq+Ex_Long_Praz) == 0:
        return None
    return At_Perm/(Patr_Liq + Ex_Long_Praz)


# Indicadores de Rentabilidade - Funções

In [ ]:
#*****************Retorno sobre o Ativo (ROA)******************************

def Ret_Ativ_ROA(df, data_ref, Denom_Cia):
    LL_A_IR_CS = Valor_conta(df, data_ref, Denom_Cia, '3.05')
    A_Total = Valor_conta(df, data_ref, Denom_Cia, '1')

    if LL_A_IR_CS is None or A_Total is None or A_Total == 0:
        return None
    return LL_A_IR_CS/A_Total


#*****************2)Retorno sobre o Investimento (ROI)******************************

def Ret_Invest_ROI(df, data_ref, Denom_Cia):
    # Investimento = Ativo Total – Passivo de Funcionamento
    
    #Ativo Total
    A_Total = Valor_conta(df, data_ref, Denom_Cia, '1')
    
    
    #2.01.01 Obrigações Sociais e Trabalhistas
    obrigST1 = Valor_conta(df, data_ref, Denom_Cia, '2.01.01')
    if obrigST1 is None:
        obrigST = 0
    else:
        obrigST = obrigST1
        
        
    #2.01.02 Fornecedores
    forn1 = Valor_conta(df, data_ref, Denom_Cia, '2.01.02')
    if forn1 is None:
        forn = 0
    else:
        forn = forn1
    #2.01.03 Obrigações Fiscais (impostos)
    obrifisc1 = Valor_conta(df, data_ref, Denom_Cia, '2.01.03')
    if obrifisc1 is None:
        obrifisc = 0
    else:
        obrifisc = obrifisc1
    #Passivo de Funcionamento
    Passivo = abs(obrigST)+abs(forn)+abs(obrifisc)
    
    Lucro = Valor_conta(df, data_ref, Denom_Cia, '3.11')
    
    if A_Total is None or Passivo is None or Lucro is None or (A_Total-Passivo) == 0:
        return None
    return Lucro/(A_Total-Passivo)



#*****************Retorno sobre o Patrimônio Líquido******************************

def Ret_PL(df, data_ref, Denom_Cia):
    Lucro = Valor_conta(df, data_ref, Denom_Cia, '3.11')
    pl1 = Valor_conta(df, data_ref, Denom_Cia, '5.01')
    pl2 = Valor_conta(df, data_ref, Denom_Cia, '5.07')
   
    if Lucro is None or pl1 is None or pl2 is None or (pl1+pl2) <= 0 or ((pl1+pl2)<=0 and Lucro<0):
        return 0
    return Lucro/((pl1 + pl2)/2)
    
    
    
  #*****************Rentabilidade das Vendas******************************

def Rent_Vendas_Liquida(df, data_ref, Denom_Cia):  
    Lucro = Valor_conta(df, data_ref, Denom_Cia, '3.11')
    Vendas = Valor_conta(df, data_ref, Denom_Cia, '3.01')
   
    if Lucro is None or Vendas is None or Vendas <= 0:
        return None
    return Lucro/Vendas 


  #*****************Margem EBITDA (OPERACIONAL)******************************
    

def M_EBITDA(df, data_ref, Denom_Cia):
    #Lucroe líquido
    Vendas = Valor_conta(df, data_ref, Denom_Cia, '3.01')
    
    # 3.05 Resultado Antes do Resultado Financeiro e dos Tributos
    RARFT = Valor_conta(df, data_ref, Denom_Cia, '3.05')

    # 7.04.01 Depreciação, Amortização e Exaustão
    DAE = Valor_conta(df, data_ref, Denom_Cia, '7.04.01')

  
            
    if Vendas is None or RARFT is None or DAE is None or Vendas <= 0:
        return None
    return (RARFT+abs(DAE))/abs(Vendas) 


  #*****************Margem OPERACIONAL******************************
    

def MarOp(df, data_ref, Denom_Cia):
    #vendas
    Vendas = Valor_conta(df, data_ref, Denom_Cia, '3.01')
    
    # 3.05 Resultado Antes do Resultado Financeiro e dos Tributos
    RARFT = Valor_conta(df, data_ref, Denom_Cia, '3.05')


  
            
    if Vendas is None or RARFT is None or Vendas <= 0:
        return None
    return (RARFT)/abs(Vendas) 


# Indicadoresde atividade para o giro de estoque e ativo

In [ ]:
#*****************Rotação dos Estoques******************************
    

def RotEst(df, data_ref, Denom_Cia):
    #1.01.04 Estoque 1.01.04
    Estoque = Valor_conta(df, data_ref, Denom_Cia, '1.01.04')
    
    # 3.02 Custos dos produtos vendidos
    CPV = Valor_conta(df, data_ref, Denom_Cia, '3.02')
    
    if Estoque is None or CPV is None or CPV == 0:
        return None
    return (Estoque)/abs(CPV)


  #*****************Rotatividade do Ativo******************************
    

def RotAtiv(df, data_ref, Denom_Cia):
    #3.01 Vendas
    Vendas = Valor_conta(df, data_ref, Denom_Cia, '3.01')
    
    # 1 Ativo
    Ativo = Valor_conta(df, data_ref, Denom_Cia, '1')
    
    if Vendas is None or Ativo is None or Ativo == 0:
        return None
    return (Vendas)/abs(Ativo)

# Indicador de Crescimento nas Vendas

In [ ]:

  #*****************Crescimento nas Vendas******************************
    

def CresVend(df, data_ref, data_refAnt, Denom_Cia):
    #3.01 Vendas
    Vendas = Valor_conta(df, data_ref, Denom_Cia, '3.01')
    VendasAnt = Valor_conta(df, data_refAnt, Denom_Cia, '3.01')
    
    if Vendas is None or VendasAnt is None or VendasAnt == 0:
        return None
    return (Vendas - VendasAnt)/abs(VendasAnt)

# Criar a base das empresas para cada ano de avaliação

In [ ]:
#Criar a base de dados dos indicadores
colunas_escolhidas3 = ['CODIGOB3','DENOM_CIA','SUBSETOR','SEGMENTO']

BD_Indicadores_2019 = base[colunas_escolhidas3].drop_duplicates().reset_index(drop=True)

BD_Indicadores_2018 = BD_Indicadores_2019.copy()
BD_Indicadores_2020 = BD_Indicadores_2019.copy()
BD_Indicadores_2021 = BD_Indicadores_2019.copy()
BD_Indicadores_2022 = BD_Indicadores_2019.copy()

# Calcular os indicadores para cada base

In [ ]:
BD_Indicadores_2018['CresVend'] = BD_Indicadores_2018.apply(lambda row: CresVend(base, '2018-12-31', '2017-12-31', row['DENOM_CIA']), axis=1)
BD_Indicadores_2018['CapTer_CapPro'] = BD_Indicadores_2018.apply(lambda row: CapTer_CapPro(base, '2018-12-31',row['DENOM_CIA']), axis=1)
BD_Indicadores_2018['CapTer_PassTot'] = BD_Indicadores_2018.apply(lambda row: CapTer_PassTot(base, '2018-12-31',row['DENOM_CIA']), axis=1)
BD_Indicadores_2018['Imob_RPer'] = BD_Indicadores_2018.apply(lambda row: Imob_RPer(base, '2018-12-31',row['DENOM_CIA']), axis=1)
BD_Indicadores_2018['LC'] = BD_Indicadores_2018.apply(lambda row: Liquidez_corrente(base, '2018-12-31',row['DENOM_CIA']), axis=1)
BD_Indicadores_2018['LS'] = BD_Indicadores_2018.apply(lambda row: Liquidez_seca(base, '2018-12-31',row['DENOM_CIA']), axis=1)
BD_Indicadores_2018['LG'] = BD_Indicadores_2018.apply(lambda row: Liquidez_geral(base, '2018-12-31',row['DENOM_CIA']), axis=1)
BD_Indicadores_2018['LI'] = BD_Indicadores_2018.apply(lambda row: liquidez_Imediata(base, '2018-12-31',row['DENOM_CIA']), axis=1)
BD_Indicadores_2018['Ret_Ativ_ROA'] = BD_Indicadores_2018.apply(lambda row: Ret_Ativ_ROA(base, '2018-12-31',row['DENOM_CIA']), axis=1)
BD_Indicadores_2018['Ret_Invest_ROI'] = BD_Indicadores_2018.apply(lambda row: Ret_Invest_ROI(base, '2018-12-31',row['DENOM_CIA']), axis=1)
BD_Indicadores_2018['Ret_PL'] = BD_Indicadores_2018.apply(lambda row: Ret_PL(base, '2018-12-31',row['DENOM_CIA']), axis=1)
BD_Indicadores_2018['RotEst'] = BD_Indicadores_2018.apply(lambda row: RotEst(base, '2018-12-31',row['DENOM_CIA']), axis=1)
BD_Indicadores_2018['RotAtiv'] = BD_Indicadores_2018.apply(lambda row: RotAtiv(base, '2018-12-31',row['DENOM_CIA']), axis=1)
BD_Indicadores_2018['Rent_Vendas_Liquida'] = BD_Indicadores_2018.apply(lambda row: Rent_Vendas_Liquida(base, '2018-12-31',row['DENOM_CIA']), axis=1)
BD_Indicadores_2018['M_EBITDA'] = BD_Indicadores_2018.apply(lambda row: M_EBITDA(base, '2018-12-31',row['DENOM_CIA']), axis=1)
BD_Indicadores_2018['MarOp'] = BD_Indicadores_2018.apply(lambda row: MarOp(base, '2018-12-31',row['DENOM_CIA']), axis=1)
BD_Indicadores_2018['Ano_Ref'] = '2018'

BD_Indicadores_2019['CresVend'] = BD_Indicadores_2019.apply(lambda row: CresVend(base, '2019-12-31', '2018-12-31', row['DENOM_CIA']), axis=1)
BD_Indicadores_2019['CapTer_CapPro'] = BD_Indicadores_2019.apply(lambda row: CapTer_CapPro(base, '2019-12-31',row['DENOM_CIA']), axis=1)
BD_Indicadores_2019['CapTer_PassTot'] = BD_Indicadores_2019.apply(lambda row: CapTer_PassTot(base, '2019-12-31',row['DENOM_CIA']), axis=1)
BD_Indicadores_2019['Imob_RPer'] = BD_Indicadores_2019.apply(lambda row: Imob_RPer(base, '2019-12-31',row['DENOM_CIA']), axis=1)
BD_Indicadores_2019['LC'] = BD_Indicadores_2019.apply(lambda row: Liquidez_corrente(base, '2019-12-31',row['DENOM_CIA']), axis=1)
BD_Indicadores_2019['LS'] = BD_Indicadores_2019.apply(lambda row: Liquidez_seca(base, '2019-12-31',row['DENOM_CIA']), axis=1)
BD_Indicadores_2019['LG'] = BD_Indicadores_2019.apply(lambda row: Liquidez_geral(base, '2019-12-31',row['DENOM_CIA']), axis=1)
BD_Indicadores_2019['LI'] = BD_Indicadores_2019.apply(lambda row: liquidez_Imediata(base, '2019-12-31',row['DENOM_CIA']), axis=1)
BD_Indicadores_2019['Ret_Ativ_ROA'] = BD_Indicadores_2019.apply(lambda row: Ret_Ativ_ROA(base, '2019-12-31',row['DENOM_CIA']), axis=1)
BD_Indicadores_2019['Ret_Invest_ROI'] = BD_Indicadores_2019.apply(lambda row: Ret_Invest_ROI(base, '2019-12-31',row['DENOM_CIA']), axis=1)
BD_Indicadores_2019['Ret_PL'] = BD_Indicadores_2019.apply(lambda row: Ret_PL(base, '2019-12-31',row['DENOM_CIA']), axis=1)
BD_Indicadores_2019['RotEst'] = BD_Indicadores_2019.apply(lambda row: RotEst(base, '2019-12-31',row['DENOM_CIA']), axis=1)
BD_Indicadores_2019['RotAtiv'] = BD_Indicadores_2019.apply(lambda row: RotAtiv(base, '2019-12-31',row['DENOM_CIA']), axis=1)
BD_Indicadores_2019['Rent_Vendas_Liquida'] = BD_Indicadores_2019.apply(lambda row: Rent_Vendas_Liquida(base, '2019-12-31',row['DENOM_CIA']), axis=1)
BD_Indicadores_2019['M_EBITDA'] = BD_Indicadores_2019.apply(lambda row: M_EBITDA(base, '2019-12-31',row['DENOM_CIA']), axis=1)
BD_Indicadores_2019['MarOp'] = BD_Indicadores_2019.apply(lambda row: MarOp(base, '2019-12-31',row['DENOM_CIA']), axis=1)
BD_Indicadores_2019['Ano_Ref'] = '2019'

BD_Indicadores_2020['CresVend'] = BD_Indicadores_2020.apply(lambda row: CresVend(base, '2020-12-31', '2019-12-31', row['DENOM_CIA']), axis=1)
BD_Indicadores_2020['CapTer_CapPro'] = BD_Indicadores_2020.apply(lambda row: CapTer_CapPro(base, '2020-12-31',row['DENOM_CIA']), axis=1)
BD_Indicadores_2020['CapTer_PassTot'] = BD_Indicadores_2020.apply(lambda row: CapTer_PassTot(base, '2020-12-31',row['DENOM_CIA']), axis=1)
BD_Indicadores_2020['Imob_RPer'] = BD_Indicadores_2020.apply(lambda row: Imob_RPer(base, '2020-12-31',row['DENOM_CIA']), axis=1)
BD_Indicadores_2020['LC'] = BD_Indicadores_2020.apply(lambda row: Liquidez_corrente(base, '2020-12-31',row['DENOM_CIA']), axis=1)
BD_Indicadores_2020['LS'] = BD_Indicadores_2020.apply(lambda row: Liquidez_seca(base, '2020-12-31',row['DENOM_CIA']), axis=1)
BD_Indicadores_2020['LG'] = BD_Indicadores_2020.apply(lambda row: Liquidez_geral(base, '2020-12-31',row['DENOM_CIA']), axis=1)
BD_Indicadores_2020['LI'] = BD_Indicadores_2020.apply(lambda row: liquidez_Imediata(base, '2020-12-31',row['DENOM_CIA']), axis=1)
BD_Indicadores_2020['Ret_Ativ_ROA'] = BD_Indicadores_2020.apply(lambda row: Ret_Ativ_ROA(base, '2020-12-31',row['DENOM_CIA']), axis=1)
BD_Indicadores_2020['Ret_Invest_ROI'] = BD_Indicadores_2020.apply(lambda row: Ret_Invest_ROI(base, '2020-12-31',row['DENOM_CIA']), axis=1)
BD_Indicadores_2020['Ret_PL'] = BD_Indicadores_2020.apply(lambda row: Ret_PL(base, '2020-12-31',row['DENOM_CIA']), axis=1)
BD_Indicadores_2020['RotEst'] = BD_Indicadores_2020.apply(lambda row: RotEst(base, '2020-12-31',row['DENOM_CIA']), axis=1)
BD_Indicadores_2020['RotAtiv'] = BD_Indicadores_2020.apply(lambda row: RotAtiv(base, '2020-12-31',row['DENOM_CIA']), axis=1)
BD_Indicadores_2020['Rent_Vendas_Liquida'] = BD_Indicadores_2020.apply(lambda row: Rent_Vendas_Liquida(base, '2020-12-31',row['DENOM_CIA']), axis=1)
BD_Indicadores_2020['M_EBITDA'] = BD_Indicadores_2020.apply(lambda row: M_EBITDA(base, '2020-12-31',row['DENOM_CIA']), axis=1)
BD_Indicadores_2020['MarOp'] = BD_Indicadores_2020.apply(lambda row: MarOp(base, '2020-12-31',row['DENOM_CIA']), axis=1)
BD_Indicadores_2020['Ano_Ref'] = '2020'


BD_Indicadores_2021['CresVend'] = BD_Indicadores_2021.apply(lambda row: CresVend(base, '2021-12-31', '2020-12-31', row['DENOM_CIA']), axis=1)
BD_Indicadores_2021['CapTer_CapPro'] = BD_Indicadores_2021.apply(lambda row: CapTer_CapPro(base, '2021-12-31',row['DENOM_CIA']), axis=1)
BD_Indicadores_2021['CapTer_PassTot'] = BD_Indicadores_2021.apply(lambda row: CapTer_PassTot(base, '2021-12-31',row['DENOM_CIA']), axis=1)
BD_Indicadores_2021['Imob_RPer'] = BD_Indicadores_2021.apply(lambda row: Imob_RPer(base, '2021-12-31',row['DENOM_CIA']), axis=1)
BD_Indicadores_2021['LC'] = BD_Indicadores_2021.apply(lambda row: Liquidez_corrente(base, '2021-12-31',row['DENOM_CIA']), axis=1)
BD_Indicadores_2021['LS'] = BD_Indicadores_2021.apply(lambda row: Liquidez_seca(base, '2021-12-31',row['DENOM_CIA']), axis=1)
BD_Indicadores_2021['LG'] = BD_Indicadores_2021.apply(lambda row: Liquidez_geral(base, '2021-12-31',row['DENOM_CIA']), axis=1)
BD_Indicadores_2021['LI'] = BD_Indicadores_2021.apply(lambda row: liquidez_Imediata(base, '2021-12-31',row['DENOM_CIA']), axis=1)
BD_Indicadores_2021['Ret_Ativ_ROA'] = BD_Indicadores_2021.apply(lambda row: Ret_Ativ_ROA(base, '2021-12-31',row['DENOM_CIA']), axis=1)
BD_Indicadores_2021['Ret_Invest_ROI'] = BD_Indicadores_2021.apply(lambda row: Ret_Invest_ROI(base, '2021-12-31',row['DENOM_CIA']), axis=1)
BD_Indicadores_2021['Ret_PL'] = BD_Indicadores_2021.apply(lambda row: Ret_PL(base, '2021-12-31',row['DENOM_CIA']), axis=1)
BD_Indicadores_2021['RotEst'] = BD_Indicadores_2021.apply(lambda row: RotEst(base, '2021-12-31',row['DENOM_CIA']), axis=1)
BD_Indicadores_2021['RotAtiv'] = BD_Indicadores_2021.apply(lambda row: RotAtiv(base, '2021-12-31',row['DENOM_CIA']), axis=1)
BD_Indicadores_2021['Rent_Vendas_Liquida'] = BD_Indicadores_2021.apply(lambda row: Rent_Vendas_Liquida(base, '2021-12-31',row['DENOM_CIA']), axis=1)
BD_Indicadores_2021['M_EBITDA'] = BD_Indicadores_2021.apply(lambda row: M_EBITDA(base, '2021-12-31',row['DENOM_CIA']), axis=1)
BD_Indicadores_2021['MarOp'] = BD_Indicadores_2021.apply(lambda row: MarOp(base, '2021-12-31',row['DENOM_CIA']), axis=1)
BD_Indicadores_2021['Ano_Ref'] = '2021'

BD_Indicadores_2022['CresVend'] = BD_Indicadores_2022.apply(lambda row: CresVend(base, '2022-12-31', '2021-12-31', row['DENOM_CIA']), axis=1)
BD_Indicadores_2022['CapTer_CapPro'] = BD_Indicadores_2022.apply(lambda row: CapTer_CapPro(base, '2022-12-31',row['DENOM_CIA']), axis=1)
BD_Indicadores_2022['CapTer_PassTot'] = BD_Indicadores_2022.apply(lambda row: CapTer_PassTot(base, '2022-12-31',row['DENOM_CIA']), axis=1)
BD_Indicadores_2022['Imob_RPer'] = BD_Indicadores_2022.apply(lambda row: Imob_RPer(base, '2022-12-31',row['DENOM_CIA']), axis=1)
BD_Indicadores_2022['LC'] = BD_Indicadores_2022.apply(lambda row: Liquidez_corrente(base, '2022-12-31',row['DENOM_CIA']), axis=1)
BD_Indicadores_2022['LS'] = BD_Indicadores_2022.apply(lambda row: Liquidez_seca(base, '2022-12-31',row['DENOM_CIA']), axis=1)
BD_Indicadores_2022['LG'] = BD_Indicadores_2022.apply(lambda row: Liquidez_geral(base, '2022-12-31',row['DENOM_CIA']), axis=1)
BD_Indicadores_2022['LI'] = BD_Indicadores_2022.apply(lambda row: liquidez_Imediata(base, '2022-12-31',row['DENOM_CIA']), axis=1)
BD_Indicadores_2022['Ret_Ativ_ROA'] = BD_Indicadores_2022.apply(lambda row: Ret_Ativ_ROA(base, '2022-12-31',row['DENOM_CIA']), axis=1)
BD_Indicadores_2022['Ret_Invest_ROI'] = BD_Indicadores_2022.apply(lambda row: Ret_Invest_ROI(base, '2022-12-31',row['DENOM_CIA']), axis=1)
BD_Indicadores_2022['Ret_PL'] = BD_Indicadores_2022.apply(lambda row: Ret_PL(base, '2022-12-31',row['DENOM_CIA']), axis=1)
BD_Indicadores_2022['RotEst'] = BD_Indicadores_2022.apply(lambda row: RotEst(base, '2022-12-31',row['DENOM_CIA']), axis=1)
BD_Indicadores_2022['RotAtiv'] = BD_Indicadores_2022.apply(lambda row: RotAtiv(base, '2022-12-31',row['DENOM_CIA']), axis=1)
BD_Indicadores_2022['Rent_Vendas_Liquida'] = BD_Indicadores_2022.apply(lambda row: Rent_Vendas_Liquida(base, '2022-12-31',row['DENOM_CIA']), axis=1)
BD_Indicadores_2022['M_EBITDA'] = BD_Indicadores_2022.apply(lambda row: M_EBITDA(base, '2022-12-31',row['DENOM_CIA']), axis=1)
BD_Indicadores_2022['MarOp'] = BD_Indicadores_2022.apply(lambda row: MarOp(base, '2022-12-31',row['DENOM_CIA']), axis=1)
BD_Indicadores_2022['Ano_Ref'] = '2022'

# Unificação das bases

In [ ]:
# Lista contendo os DataFrames
dataframes = [BD_Indicadores_2018, BD_Indicadores_2019, BD_Indicadores_2020, BD_Indicadores_2021, BD_Indicadores_2022]

# Concatenação dos DataFrames
df_concatenado = pd.concat(dataframes)